In [ ]:
spark

In [ ]:
import pandas as pd
from google.cloud import storage
from io import BytesIO
from datetime import datetime, date
import holidays

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, udf, to_date, year, month, date_format, size, split, dayofweek
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler, MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline




In [ ]:
bucket_name = 'my-bigdataproject-jg'
gs_path  = f'gs://{bucket_name}/'
cleaned_folder = 'cleaned/'
destination_folder = 'code_and_models/'

storage_client = storage.Client() 
bucket = storage_client.get_bucket(bucket_name)

In [ ]:
spark = SparkSession.builder.master("local[*]") \
    .appName("Taxi Demand Prediction") \
    .getOrCreate()

In [ ]:
weather_files = gs_path + cleaned_folder + 'weather_data_*.parquet'
weather_df = spark.read.parquet(weather_files)
weather_df.show(15)

In [ ]:
taxi_files = gs_path + cleaned_folder + "cleaned_yellow_tripdata_2023-??.parquet"
taxi_df = spark.read.parquet(taxi_files)
taxi_df.show(15)

In [ ]:
taxi_df = taxi_df.drop('__index_level_0__')
taxi_df = taxi_df.drop('dropoff_datetime')
taxi_df = taxi_df.drop('DOLocationID')
taxi_df = taxi_df.withColumn('pickup_datetime', to_date(col('pickup_datetime')))

taxi_df.show(15)

In [ ]:
taxi_zone_file = gs_path + cleaned_folder + 'taxi_zones_data.csv'
taxi_zone_df = spark.read.parquet(taxi_zone_file)

taxi_df.show(15)

In [ ]:
taxi_df = taxi_df.join(taxi_zone_df, taxi_df.PULocationID == taxi_zone_df.LocationID)
taxi_df = taxi_df.drop('zone')
taxi_df = taxi_df.drop('PULocationID')
taxi_df = taxi_df.drop('LocationID')
taxi_df.show(15)

In [ ]:
taxi_df.createOrReplaceTempView('taxi_df')
taxi_df = spark.sql('SELECT pickup_datetime, borough, COUNT(*) as total_trips FROM taxi_df GROUP BY pickup_datetime, borough')
taxi_df = taxi_df.dropna(subset=['total_trips'])
taxi_df = taxi_df.filter(taxi_df.borough != 'EWR')
taxi_df.show(15)

In [ ]:
weather_df = weather_df.withColumnRenamed('borough', 'weather_borough')

combined_df = taxi_df.join(weather_df, [taxi_df.pickup_datetime == weather_df.datetime, taxi_df.borough == weather_df.weather_borough])

combined_df = combined_df.drop('pickup_datetime').
combined_df = combined_df.drop('weather_borough')

combined_df.show(15)

In [ ]:
"""
# Feature Engineering
Original column - 

Features:

datetime

Features:
Month of the Year
Day of the Week 
Weekend
Holiday

borough - StringIndexer

temp
 - Use the lowest tempmin value and highest tempmax value to create a range for min max scaling
 - MinMaxScaler

feels_like
- Use the lowest tempmin value and highest tempmax value to create a range for min max scaling
- MinMaxScaler

use min max scaling for the following columns:
humidity
precip
snow
snowdepth
windspeed
cloudcover
visibility

uvindex - Encode directly as integer

conditions
 - Conditions has these options Clear, Snow, Overcast, Rain, Partially cloudy, Overcast
 - Some records have these options joint, for example: Snow, Rain, Overcast |      Rain, Overcast
 - split these options into separate columns and encode them as binary values
"""
# datetime
combined_df = combined_df.withColumn('month', month(col('datetime')))
combined_df = combined_df.withColumn('dayofweek', dayofweek(col('datetime')))
combined_df = combined_df.withColumn('weekend', when(col('dayofweek') == 1, 1.0).when(col('dayofweek') == 7, 1.0).otherwise(0))

# temp

temp_min = combined_df.agg(F.min('tempmin')).collect()[0][0]
temp_max = combined_df.agg(F.max('tempmax')).collect()[0][0]

temp_assembler = VectorAssembler(inputCols=['temp'], outputCol='temp_vector')
temp_scaler = MinMaxScaler(inputCol='temp_vector', outputCol='temp_scaled', min=temp_min, max=temp_max)

# feels_like
feelslike_min = combined_df.agg(F.min('feelslikemin')).collect()[0][0]
feelslike_max = combined_df.agg(F.max('feelslikemax')).collect()[0][0]

feelslike_assembler = VectorAssembler(inputCols=['feelslike'], outputCol='feelslike_vector')
feelslike_scaler = MinMaxScaler(inputCol='feelslike_vector', outputCol='feelslike_scaled', min=feelslike_min, max=feelslike_max)

# humidity
humidity_assembler = VectorAssembler(inputCols=['humidity'], outputCol='humidity_vector')
humidity_scaler = MinMaxScaler(inputCol='humidity_vector', outputCol='humidity_scaled')

# precip
precip_assembler = VectorAssembler(inputCols=['precip'], outputCol='precip_vector')
precip_scaler = MinMaxScaler(inputCol='precip_vector', outputCol='precip_scaled')

# snow
snow_assembler = VectorAssembler(inputCols=['snow'], outputCol='snow_vector')
snow_scaler = MinMaxScaler(inputCol='snow_vector', outputCol='snow_scaled')

# snowdepth
snowdepth_assembler = VectorAssembler(inputCols=['snowdepth'], outputCol='snowdepth_vector')
snowdepth_scaler = MinMaxScaler(inputCol='snowdepth_vector', outputCol='snowdepth_scaled')

# windspeed
windspeed_assembler = VectorAssembler(inputCols=['windspeed'], outputCol='windspeed_vector')
windspeed_scaler = MinMaxScaler(inputCol='windspeed_vector', outputCol='windspeed_scaled')

# cloudcover
cloudcover_assembler = VectorAssembler(inputCols=['cloudcover'], outputCol='cloudcover_vector')
cloudcover_scaler = MinMaxScaler(inputCol='cloudcover_vector', outputCol='cloudcover_scaled')

# visibility
visibility_assembler = VectorAssembler(inputCols=['visibility'], outputCol='visibility_vector')
visibility_scaler = MinMaxScaler(inputCol='visibility_vector', outputCol='visibility_scaled')

# uvindex

# conditions






In [ ]:
indexer = StringIndexer(inputCol='borough', outputCol='borough_index')
encoder = OneHotEncoder(inputCols=['borough_index', 'month', 'dayofweek', 'weekend'], outputCols=['borough_Vector', 'month_Vector', 'dayofweek_Vector', 'weekend_Vector'])
assembler = VectorAssembler(inputCols=['month_Vector', 'dayofweek_Vector', 'weekend_Vector', 'borough_Vector', 'temp_scaled', 'feelslike_scaled', 'humidity_scaled', 'precip_scaled', 'snow_scaled', 'snowdepth_scaled', 'windspeed_scaled', 'cloudcover_scaled', 'visibility_scaled', 'uvindex'], outputCol='features')

In [ ]:
# Create a new frame with the transformed data
pipeline = Pipeline(stages=[temp_assembler, temp_scaler, feelslike_assembler, feelslike_scaler, humidity_assembler, humidity_scaler, precip_assembler, precip_scaler, snow_assembler, snow_scaler, snowdepth_assembler, snowdepth_scaler, windspeed_assembler, windspeed_scaler, cloudcover_assembler, cloudcover_scaler, visibility_assembler, visibility_scaler, indexer, encoder, assembler])
model = pipeline.fit(combined_df)
transformed_df = model.transform(combined_df)

# Save the transformed data
transformed_df.write.parquet(gs_path + destination_folder + 'features')

In [ ]:
train_df, test_df = combined_df.randomSplit([0.7, 0.3], seed=42)

linear_reg = LinearRegression(labelCol='total_trips')

evaluator = RegressionEvaluator(labelCol='total_trips')

regresion_pipe = Pipeline(stages=[indexer, temp_assembler, temp_scaler, feelslike_assembler, feelslike_scaler, humidity_assembler, humidity_scaler, precip_assembler, precip_scaler, snow_assembler, snow_scaler, snowdepth_assembler, snowdepth_scaler, windspeed_assembler, windspeed_scaler, cloudcover_assembler, cloudcover_scaler, visibility_assembler, visibility_scaler, encoder, assembler, linear_reg])

grid = ParamGridBuilder()
grid = grid.build()

cv = CrossValidator(estimator=regresion_pipe, estimatorParamMaps=grid, evaluator=evaluator, numFolds=3)

all_models = cv.fit(train_df)

print(f"Average metrics: {all_models.avgMetrics}")

best_model = all_models.bestModel

test_results = best_model.transform(test_df)

rmse = evaluator.evaluate(test_results, {evaluator.metricName: "rmse"})
mse = evaluator.evaluate(test_results, {evaluator.metricName: "mse"})
mae = evaluator.evaluate(test_results, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(test_results, {evaluator.metricName: "r2"})

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R2: {r2}")

In [ ]:
coefficients = best_model.stages[-1].coefficients
print("bestModel coefficients", coefficients)

In [ ]:
# Save the model
model_path = gs_path + destination_folder + 'model'
best_model.write().overwrite().save(model_path)

In [ ]:
indexer = StringIndexer(inputCol='borough', outputCol='borough_index')
encoder = OneHotEncoder(inputCol='borough_index', outputCol='borough_encoded')
assembler = VectorAssembler(inputCols=['month', 'dayofweek', 'weekend', 'borough_encoded', 'temp_scaled', 'feelslike_scaled', 'humidity_scaled', 'precip_scaled', 'snow_scaled', 'snowdepth_scaled', 'windspeed_scaled', 'cloudcover_scaled', 'visibility_scaled', 'uvindex'], outputCol='features')

train_df, test_df = combined_df.randomSplit([0.7, 0.3], seed=42)

linear_reg = LinearRegression(labelCol='total_trips')

evaluator = RegressionEvaluator(labelCol='total_trips')

regresion_pipe = Pipeline(stages=[indexer, encoder, temp_assembler, temp_scaler, feelslike_assembler, feelslike_scaler, humidity_assembler, humidity_scaler, precip_assembler, precip_scaler, snow_assembler, snow_scaler, snowdepth_assembler, snowdepth_scaler, windspeed_assembler, windspeed_scaler, cloudcover_assembler, cloudcover_scaler, visibility_assembler, visibility_scaler, assembler, linear_reg])

grid = ParamGridBuilder()
grid = grid.build()
cv = CrossValidator(estimator=regresion_pipe, estimatorParamMaps=grid, evaluator=evaluator, numFolds=3)
all_models = cv.fit(train_df)

print(f"Average metrics: {all_models.avgMetrics}")

best_model = all_models.bestModel

test_results = best_model.transform(test_df)

rmse = evaluator.evaluate(test_results, {evaluator.metricName: "rmse"})
mse = evaluator.evaluate(test_results, {evaluator.metricName: "mse"})
mae = evaluator.evaluate(test_results, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(test_results, {evaluator.metricName: "r2"})

print(f"RMSE: {rmse}")
print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"R2: {r2}")

In [ ]:
indexer = StringIndexer(inputCol='borough', outputCol='borough_index')
encoder = OneHotEncoder(inputCol='borough_index', outputCol='borough_encoded')
assembler = VectorAssembler(inputCols=['month', 'dayofweek', 'weekend', 'borough_encoded', 'temp_scaled', 'feelslike_scaled', 'humidity_scaled', 'precip_scaled', 'snow_scaled', 'snowdepth_scaled', 'windspeed_scaled', 'cloudcover_scaled', 'visibility_scaled', 'uvindex'], outputCol='features')

pipeline = Pipeline(stages=[temp_assembler, temp_scaler, feelslike_assembler, feelslike_scaler, humidity_assembler, humidity_scaler, precip_assembler, precip_scaler, snow_assembler, snow_scaler, snowdepth_assembler, snowdepth_scaler, windspeed_assembler, windspeed_scaler, cloudcover_assembler, cloudcover_scaler, visibility_assembler, visibility_scaler, indexer, encoder, assembler])

pipeline_model = pipeline.fit(combined_df).transform(combined_df)

train_df, test_df = pipeline_model.randomSplit([0.7, 0.3], seed=42)

lr = LinearRegression(featuresCol='features', labelCol='total_trips')
lr_model = lr.fit(train_df)

lr_predictions = lr_model.transform(test_df)
lr_evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='total_trips')

rmse = lr_evaluator.evaluate(lr_predictions, {lr_evaluator.metricName: 'rmse'})
mae = lr_evaluator.evaluate(lr_predictions, {lr_evaluator.metricName: 'mae'})
mse = lr_evaluator.evaluate(lr_predictions, {lr_evaluator.metricName: 'mse'})
r2 = lr_evaluator.evaluate(lr_predictions, {lr_evaluator.metricName: 'r2'})

print(f'RMSE: {rmse}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2: {r2}')

